In [1]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pandas as pd

%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Creacion de constantes
# El valor que retorna el constructos es el valor de la constante

# Creamos constantes a=2 y b=3
a = tf.constant(2)
b = tf.constant(3)
# Creamos matrices de 3x3

matriz1 = tf.constant([[1,3,2], [1,0,0], [1,2,2]])
matriz2 = tf.constant([[1,0,5], [7,5,0],[2,1,1]])

In [7]:
# Realizamos algunos cáculos con estas constantes
suma = tf.add(a,b)
mult = tf.multiply(a,b)
cubo_a = a**3

# suma de matrices 
suma_mat = tf.add(matriz1, matriz2)

# producto de matrices
mult_mat = tf.matmul(matriz1, matriz2)

In [9]:
# Todo en tensorflowocurre dentro de una sesión

# creamos la sesion y realizamos algunas operaciones con las constantes
# y lanzamos la sesión

with tf.Session() as sess:
    print("Suma de las constantes: {}".format(sess.run(suma)))
    print("Multiplicación de las constantes: {}". format(sess.run(mult)))
    print("Constante elevada al cubo: {}".format(sess.run(cubo_a)))
    print("Suma de matrices: \n{}".format(sess.run(suma_mat)))
    print("Producto de matrices: \n{}".format(sess.run(mult_mat)))

Suma de las constantes: 5
Multiplicación de las constantes: 6
Constante elevada al cubo: 8
Suma de matrices: 
[[2 3 7]
 [8 5 0]
 [3 3 3]]
Producto de matrices: 
[[26 17  7]
 [ 1  0  5]
 [19 12  7]]


In [11]:
# Variables persistentes

# creamos una variable y la inicializamos con 0
estado = tf.Variable(0, name="contador")

# Creamos la op que le va a sumar uno a la Variable estado

uno = tf.constant(1)
nuevo_valor = tf.add(estado, uno)
actualizar = tf.assign(estado, nuevo_valor)

# Las variables deben ser inicializadas por la operacion init luego de 
# lanzar el grafo. Debemos agregar la op init a nuestro grafo
init = tf.global_variables_initializer()

# lanzamos la sesion y ejecutamos las operaciones
with tf.Session() as sess:
    # Ejecutamos la op init
    sess.run(init)
    #imprimir el valor de la variable estado
    print(sess.run(estado))
    # ejcutamos la op que va a actualizar a estado
    for _ in range(3):
        sess.run(actualizar)
        print(sess.run(estado))

0
1
2
3


In [14]:
# variables simbólicas (contenedores)
# ejemplo variables simbolicas en los grafos
# el valor que ddevuelve el constructor representa la salida de la 
# variable (la entrada de la variable se define en la sesion)

# creamos un contenedor del tipo float. Un tensor de 4x4
x = tf.placeholder(tf.float32, shape=(4,4))
y = tf.matmul(x, x)

with tf.Session() as sess:
    # print(sess.run(y)) # Error:va a fallar porque no alimentamos a x
    rand_array = np.random.rand(4, 4)
    print(sess.run(y, feed_dict={x: rand_array})) # ahora esta correcto

[[1.125122   0.83877337 0.9310026  1.0142438 ]
 [1.0867037  0.6419584  0.60527205 0.7598926 ]
 [0.721156   0.5856165  0.7069878  0.6357784 ]
 [0.7691011  0.65283394 0.5084772  0.6175576 ]]


In [15]:
# ejemplo de neuronas simples 


# Neurona con tensorflow
# Defino las entradas 
entradas = tf.placeholder("float", name='Entradas')
datos = np.array([[0,0],[1,0],[0,1],[1,1]])

# defino las salidas 
uno = lambda: tf.constant(1.0)
cero = lambda: tf.constant(0.0)

with tf.name_scope('Pesos'):
    # Definiendo pesos y sesgo
    pesos = tf.placeholder("float", name="Pesos")
    sesgo = tf.placeholder("float", name='Sesgo')

with tf.name_scope('Activacion'):
    # Funcion de activacion
    activacion = tf.reduce_sum(tf.add(tf.matmul(entradas,pesos), sesgo))
    
with tf.name_scope('Neurona'):
    # defino la neurona
    def neurona():
        return tf.case([(tf.less(activacion, 0.0), cero)], default=uno)
    
    # salida
    a = neurona()
# path de logs
logs_path = '/tmp/tensorflow_logs/neurona'

In [22]:
# lanzar la Sesion
with tf.Session() as sess:
    # para armar el grafo
    summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)
    
    # para armar la tabla de verdad
    x_1 = []
    x_2 = []
    out = []
    act = []
    
    for i in range(len(datos)):
        t = datos[i].reshape(1,2)
        salida, activ = sess.run([a, activacion], feed_dict={entradas: t, pesos:np.array([[1.],[1.]]), sesgo: -1.5})
        
        # armar tabla de verdad en DataFrame
        x_1.append(t[0][0])
        x_2.append(t[0][1])
        out.append(salida)
        act.append(activ)
        
    tabla_info = np.array([x_1, x_2, act, out]).transpose()
    tabla = pd.DataFrame(tabla_info, columns=['x1', 'x2', 'f(x)', 'x1 AND x2'])
    
    print(tabla)
    """
    Aquí podemos ver los datos de entrada de x1 y x2, el resultado de la función de activación y la decisión final 
    que toma la neurona de acuerdo este último resultado. Como podemos ver en la tabla de verdad, 
    la neurona nos dice que x1 and x2 solo es verdad cuando ambos son verdaderos, lo que es correcto.
    """

    x1   x2  f(x)  x1 AND x2
0  0.0  0.0  -1.5        0.0
1  1.0  0.0  -0.5        0.0
2  0.0  1.0  -0.5        0.0
3  1.0  1.0   0.5        1.0


In [23]:
# Nuerona OR
"""
salida de neurona => a = f(-0.5+x1+x2)
"""

#Neurona OR, solo cambiamos el valor del sesgo
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)
    
    # para armar la tabla
    x_1 = []
    x_2 = []
    out = []
    act = []
    
    for i in range(len(datos)):
        t = datos[i].reshape(1,2)
        salida, activ = sess.run([a, activacion], feed_dict={entradas: t, pesos:np.array([[1.],[1.]]), sesgo: -0.5}) # sesgo ahora -0.5
        
        # armar tabla de verdad en DataFrame
        x_1.append(t[0][0])
        x_2.append(t[0][1])
        out.append(salida)
        act.append(activ)
        
    tabla_info = np.array([x_1, x_2, act, out]).transpose()
    tabla = pd.DataFrame(tabla_info, columns=['x1', 'x2', 'f(x)', 'x1 AND x2'])
    
    print(tabla)

    x1   x2  f(x)  x1 AND x2
0  0.0  0.0  -0.5        0.0
1  1.0  0.0   0.5        1.0
2  0.0  1.0   0.5        1.0
3  1.0  1.0   1.5        1.0
